# Comparación condiciones cara y no-cara

Importar todas las librerías necesarias para realizar los cálculos

In [1]:
import glob
import logging
import numpy as np
import os
import json

from scipy.io import wavfile
import scipy.stats as stats
import scipy.signal as snl
import processing as pr

logging.disable(logging.CRITICAL)

 2020-11-18 12:37:08,783 - DEBUG - Define classes and functions.


Definir las constantes del experimento que se van a usar en el análisis

In [2]:
freq_ranges = {"delta": (1, 3), "theta_low": (3, 5), "theta_high": (5, 7),
               "alpha": (7, 12), "beta": (12, 30), "gamma_low": (30, 50), "gamma_high": (50, 100)}
bands = list(freq_ranges.keys())
freq_limits = [1, 3, 5, 7, 12, 30, 50, 100]
channels = ['Fp1', 'Fpz', 'Fp2', 'AF7', 'AF3', 'AF4', 'AF8', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'HEOG+', 'HEOG', 'VEOG+', 'VEOG', 'M1']
phs_lims = np.linspace(-np.pi, np.pi, 4)

Definir los lugares donde se encuentran los archivos

In [3]:
raw_data_folder_path = "../raw_data/"
segment_folder_path = "../preprocessed_data/segments/"
segment_file_paths = glob.glob(segment_folder_path + "*.dat")
mi_folder_path = "../results/mutualinfo/ds8/"
mi_file_paths = glob.glob(mi_folder_path + "*.mi")
multimedia_folder_path = "../raw_data/Stimuli/"  # "../Data/downsampled_audios/"
audio_file_paths = glob.glob(multimedia_folder_path + "*.wav")

Calcular la duración mínima y máxima de los audios

In [4]:
duracion = []
for audio in audio_file_paths:
    freq, data = wavfile.read(audio)
    duracion.append(len(data)/freq)
duracion.sort()
print("Rango de duraciones de los audios: entre {:.2f}s y {:.2f}s, con media de {:.1f} y desviación std de {:.1f}".format(duracion[0], duracion[-1], np.mean(duracion), np.std(duracion)))

/Library/Python/3.7/site-packages/ipykernel_launcher.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  This is separate from the ipykernel package so we can avoid doing imports until


Rango de duraciones de los audios: entre 2.19s y 5.09s, con media de 3.3 y desviación std de 0.4


## Abrir los archivos de MI ya calculados (usando MI_calculation.py)

Abrir todos los archivos de Mutual Information y añadir sus valores a la varias variables adecuadas en base al **criterio que usaremos para el estudio**.
En este caso utilizamos cara vs no-cara.

Definimos mutual_information_sf que nos va a servir para las estadísticas y cumdata para los Kruskall-Wallis.

In [36]:
mutual_information_sf = {"correct": {}, "incorrect": {}, "incorrect_n": 0, "correct_n": 0}
cumdata = {}

for file in mi_file_paths:
    mi = pr.MutualInformation()
    mi.load(file)
    if True:
        if mi.audio_corr:
            kw = "correct"
        else:
            kw = "incorrect"
        kwn = kw + "_n"
        mutual_information_sf[kwn] += 1
        if mutual_information_sf[kw] == {}:
            for band in bands:
                mutual_information_sf[kw][band] = {}
                for channel in channels:
                    mutual_information_sf[kw][band][channel] = float(mi.mi[band][channel])
        else:
            for band in bands:
                for channel, value in mi.mi[band].items():
                    mutual_information_sf[kw][band][channel] += float(value)
                    
        if cumdata == {}:
            for band in bands:
                cumdata[band] = {}
                for channel in channels:
                    cumdata[band][channel] = {"correct": [], "incorrect": []}
                
        for band in bands:
            for channel in channels:
                if mi.audio_corr:
                    cumdata[band][channel]["correct"].append(float(mi.mi[band][channel]))
                else:
                    cumdata[band][channel]["incorrect"].append(float(mi.mi[band][channel]))

# Statistical analysis

### Kolmogorov-Smirnov test

In [14]:
ks_test = {}
for band in bands:
    ks_test[band] = {}
    for channel in channels:
        ks_test[band][channel] = {}
        for state in ["correct", "incorrect"]:
            ks_test[band][channel][state] = stats.kstest(cumdata[band][channel][state], 'norm')

In [17]:
values = []
for band in ks_test.keys():
    for channel in ks_test[band].keys():
        values.append(ks_test[band][channel]["correct"][0])

In [18]:
print("$" + str(np.mean(values)) + " \\pm " + str(np.std(values)) + "$")

$0.9992376560265298 \pm 7.079813346328179e-08$


#### Advanced stats for imshow

In [ ]:
dimensions = np.array([len(kw_test["delta"].keys()), len(kw_test.keys()), 2]) # Rows are channels and columns are bands
kw_values = np.zeros(dimensions)

In [ ]:
bands = list(kw_test.keys())
channels = list(kw_test["delta"].keys())

In [ ]:
for band in range(len(ks_test.keys())):
    for channel in range(len(ks_test[bands[band]].keys())):
        kw_values[channel, band, :] = (np.array(kw_test[bands[band]][channels[channel]])[0], \
                                       np.array(kw_test[bands[band]][channels[channel]])[1])

In [ ]:
logging.disable(logging.CRITICAL)

In [ ]:
print("\\hline \n & $" + "$ & $".join(bands) + "$ \\\\ \n\\hline")
for channel in range(len(channels)):
    print_values = []
    for band in range(len(bands)):
        #string_value = kw_values[channel,band,0]
        #p = kw_values[channel, band, 1]
        #if p >= 0.01:
            #string_value = "{:0.2f}".format(p) #+ "{"
        #elif p >= 0.001:
            #string_value =  "{:0.3f}".format(p)#+ "{"
        #else:
            #string_value = "\\bm{" + "<0.001" + "}"
            #string_value = "{:0.1e}".format(p).replace("e", "\\times 10^{")
        #print_values.append(string_value)
        print_values.append("{:2.3f}".format(string_value))
    print(channels[channel] + " & $" + "$ & $".join(print_values) + "$ \\\\")


#### Most notable ones

In [ ]:
print("\\hline \n Banda & Canal & H-valor & P-valor \\\\ \n\\hline")
for band in range(len(bands)):
    for channel in range(len(channels)):
        p = kw_values[channel, band, 1]
        if p <= 0.01:
            if p <= 0.001:
                string_value = "<0.001"
            else:
                string_value = "{:0.3f}".format(p)
            print("$ \\" + bands[band] + " $ & " + channels[channel] + " & $" + \
                 "{:0.2f}".format(kw_values[channel, band, 0]) + \
                 "$ & $" + string_value + "$ \\\\")

## Kruskal-Wallis analysis

In [43]:
print("Bonferroni correction factor: {}".format(len(bands)*len(channels)))
alpha = 0.9
saved = []
fandp_kruskal = {}
for band in bands:
    fandp_kruskal[band] = {}
    for channel in channels:
        fandp_kruskal[band][channel] = list(stats.kruskal(cumdata[band][channel]["correct"], cumdata[band][channel]["incorrect"]))
        fandp_kruskal[band][channel][1] *= len(bands)*len(channels)
        if fandp_kruskal[band][channel][1] < alpha:
            print("Channel {} in band {} is significant below {} ({:.1e}) with H value {:2.2f}".format(channel, band, alpha, fandp_kruskal[band][channel][1], fandp_kruskal[band][channel][0]))
            saved.append("{} {}".format(band, channel))

Bonferroni correction factor: 448
Channel Pz in band delta is significant below 0.9 (8.7e-01) with H value 9.61


In [25]:
saved

['gamma_low Fpz', 'gamma_low Fz', 'gamma_high Fz']

## MI differences and relative differences

In [23]:
with open("./MI_diff_rel.json") as jsonfile:
    mi = json.load(jsonfile)

In [33]:
for elem in saved:
    band = elem.split(" ")[0]
    channel = elem.split(" ")[1]
    diff = mi["MI_diff"][band][channel]
    rel = mi["MI_rel"][band][channel]
    kw = fandp_kruskal[band][channel]
    print("{}, {}, {:1.1e}, {:1.1e}, {:3.2f}, {:1.1e}".format(band, channel, diff, rel, kw[0], kw[1]))

gamma_low, Fpz, -2.5e-03, 8.0e-04, 25.02, 2.5e-04
gamma_low, Fz, -3.0e-03, 9.5e-04, 28.03, 5.3e-05
gamma_low, F2, -1.6e-03, 4.9e-04, 17.20, 1.5e-02
gamma_high, Fpz, -2.6e-03, 8.2e-04, 19.50, 4.5e-03


In [24]:
#print("\\hline \n & $" + "$ & $".join(bands) + "$ \\\\ \n\\hline")
for channel in mi["MI_rel"]["delta"].keys():
    print_values = []
    for band in freq_ranges.keys():
        value = mi["MI_rel"][band][channel]
        if value >= 0.001:
            string_value = "{:0.3f}".format(value) + " {"
        else:
            string_value = "{:0.1e}".format(value).replace("e", " \\times 10^{")

        print_values.append(string_value)
    print(channel + " & $" + "}$ & $".join(print_values) + "}$ \\\\")

Fp1 & $-1.3 \times 10^{-03}$ & $-3.8 \times 10^{-04}$ & $6.8 \times 10^{-07}$ & $2.1 \times 10^{-05}$ & $6.3 \times 10^{-05}$ & $5.5 \times 10^{-05}$ & $8.1 \times 10^{-05}$ \\
Fpz & $-6.8 \times 10^{-04}$ & $2.9 \times 10^{-04}$ & $6.2 \times 10^{-05}$ & $1.1 \times 10^{-04}$ & $6.2 \times 10^{-05}$ & $8.0 \times 10^{-04}$ & $8.2 \times 10^{-04}$ \\
Fp2 & $-7.5 \times 10^{-04}$ & $2.0 \times 10^{-04}$ & $2.7 \times 10^{-04}$ & $5.4 \times 10^{-05}$ & $1.0 \times 10^{-04}$ & $1.5 \times 10^{-04}$ & $1.8 \times 10^{-04}$ \\
AF7 & $-2.1 \times 10^{-03}$ & $-1.3 \times 10^{-04}$ & $6.5 \times 10^{-05}$ & $1.2 \times 10^{-04}$ & $3.6 \times 10^{-05}$ & $-1.1 \times 10^{-05}$ & $6.9 \times 10^{-06}$ \\
AF3 & $-1.4 \times 10^{-03}$ & $-1.9 \times 10^{-04}$ & $1.2 \times 10^{-04}$ & $5.0 \times 10^{-05}$ & $-8.4 \times 10^{-06}$ & $5.5 \times 10^{-04}$ & $6.5 \times 10^{-04}$ \\
AF4 & $-2.0 \times 10^{-04}$ & $-2.6 \times 10^{-04}$ & $5.7 \times 10^{-05}$ & $-6.9 \times 10^{-05}$ & $8.7 \time

44.42% son negativos

## MI Changes through bands

In [ ]:
for key in ["face", "scrambled"]:
    for band in bands:
        for channel in channels:
            try:
                mutual_information_sf[key][band][channel] = list(map(float, mutual_information_sf[key][band][channel]))
            except ValueError:
                print("A")

In [ ]:
mutual_information_sf.keys()

In [ ]:
MI_diff = {}
MI_rel = {}
for band in mutual_information_sf["face"].keys():
    MI_diff[band] = {}
    MI_rel[band] = {}
    for channel, value in mutual_information_sf["face"][band].items():
        MI_diff[band][channel] = value - \
            mutual_information_sf["scrambled"][band][channel]
        MI_rel[band][channel] = MI_diff[band][channel] / value

In [ ]:
bands = list(freq_ranges.keys())
channels = list(mi["MI_diff"]["delta"].keys())

In [ ]:
means = {}
std = {}
valuesd = {}
for band in bands:
    values = []
    valuesd[band] = []
    for channel in channels:
        valuesd[band].append(mi["MI_diff"][band][channel])
        values.append(np.abs(mi["MI_diff"][band][channel]))
    means[band] = np.mean(values)
    std[band] = np.std(values)

In [ ]:
y = list(means.values())
ticks = list(means.keys())

In [ ]:
xx = []
for item in valuesd:
    xx.append(item)

In [ ]:
fig, ax = plt.subplots(figsize=(16,12))
ax.boxplot(valuesd.values())
yticks = ax.get_yticklabels()
ax.set_xticklabels([r"$\delta$",r"$\theta_{low}$", r"$\theta_{high}$", r"$\alpha$", r"$\beta$", r"$\gamma$", r"Altas frec." ], fontsize = 20)
ax.set_ylabel(r"$\Delta$", fontsize = 30)
ax.set_xlabel(r"Bandas de frecuencia", fontsize = 30)
ax.set_title(r"Comparación de los rangos de valores de $\Delta$ en las distintas bandas", fontsize = 25)
#ax.set_yticklabels(list(yticks), fontsize=30)